In [1]:
import pandas as pd
from datetime import datetime, timedelta
import pytz

In [2]:
with open ("8K_github_username.txt", "r", encoding="utf8") as f:
    all_username = f.readlines()

In [3]:
with open ("not_a_user_or_no_data.txt", "r", encoding="utf8") as f:
    not_a_user_or_no_data = f.readlines()

In [4]:
for user in not_a_user_or_no_data:
    all_username.remove(user)

In [5]:
all_username = [x.strip() for x in all_username]

In [6]:
profile_df = pd.read_csv("all_user_profile_data.csv")

since github rest api keeps user event data for 90 days, I'm going to remove user without event data by querying in "updated_at" column using pandas.

In [7]:
print("today: ",datetime.now().day, '/', datetime.now().month, '/', datetime.now().year)

today:  14 / 1 / 2024


In [8]:
profile_df.head()

,user_alias,type,hireable,location,followers,following,public_repos,public_gists,created_at,updated_at
0,usr0,User,NaN,NaN,2.0,7.0,43.0,NaN,2020-09-07T23:54:51Z,2023-10-05T18:16:01Z
1,usr1,User,NaN,Tbilisi,3.0,4.0,16.0,NaN,2011-04-16T12:30:37Z,2024-01-08T13:45:16Z
2,usr2,User,NaN,Karachi,13.0,23.0,44.0,NaN,2023-01-03T13:56:42Z,2023-12-24T05:10:36Z
3,usr3,User,True,Karachi,NaN,NaN,5.0,NaN,2021-09-24T09:04:40Z,2023-05-02T19:11:00Z
4,usr4,User,NaN,NaN,1.0,NaN,2.0,NaN,2022-10-07T11:01:08Z,2023-12-07T20:03:36Z


In [9]:
profile_df['updated_at'] = pd.to_datetime(profile_df['updated_at'])
profile_df.head()

,user_alias,type,hireable,location,followers,following,public_repos,public_gists,created_at,updated_at
0,usr0,User,NaN,NaN,2.0,7.0,43.0,NaN,2020-09-07T23:54:51Z,2023-10-05 18:16:01+00:00
1,usr1,User,NaN,Tbilisi,3.0,4.0,16.0,NaN,2011-04-16T12:30:37Z,2024-01-08 13:45:16+00:00
2,usr2,User,NaN,Karachi,13.0,23.0,44.0,NaN,2023-01-03T13:56:42Z,2023-12-24 05:10:36+00:00
3,usr3,User,True,Karachi,NaN,NaN,5.0,NaN,2021-09-24T09:04:40Z,2023-05-02 19:11:00+00:00
4,usr4,User,NaN,NaN,1.0,NaN,2.0,NaN,2022-10-07T11:01:08Z,2023-12-07 20:03:36+00:00


In [10]:
_90_days_ago = datetime(2024, 1, 14, tzinfo=pytz.UTC) - timedelta(days=90)

In [11]:
filtered_df = profile_df[profile_df['updated_at'] > _90_days_ago]

In [12]:
filtered_df.head()

,user_alias,type,hireable,location,followers,following,public_repos,public_gists,created_at,updated_at
1,usr1,User,NaN,Tbilisi,3.0,4.0,16.0,NaN,2011-04-16T12:30:37Z,2024-01-08 13:45:16+00:00
2,usr2,User,NaN,Karachi,13.0,23.0,44.0,NaN,2023-01-03T13:56:42Z,2023-12-24 05:10:36+00:00
4,usr4,User,NaN,NaN,1.0,NaN,2.0,NaN,2022-10-07T11:01:08Z,2023-12-07 20:03:36+00:00
5,usr5,User,True,Remote,NaN,6.0,22.0,7.0,2022-06-02T04:04:01Z,2023-12-21 10:06:23+00:00
6,usr6,User,True,NaN,1.0,2.0,19.0,NaN,2021-08-20T19:03:28Z,2023-12-18 08:21:10+00:00


In [13]:
print("before: ", len(profile_df), "\nafter: ", len(filtered_df))

before:  8583 
after:  5844


In [14]:
filtered_username = filtered_df['user_alias'].values.tolist()

In [15]:
username_dict = {}

In [16]:
username_alias = ["usr"+str(x) for x in range(len(all_username))]

In [17]:
for i in range(len(username_alias)):
    username_dict[username_alias[i]] = all_username[i]

In [18]:
amount_of_username_in_each_dataset = len(filtered_username)/28
amount_of_username_in_each_dataset

208.71428571428572

In [19]:
amount_of_request_time_in_each_dataset = amount_of_username_in_each_dataset/60
print("min: ", amount_of_request_time_in_each_dataset, "hrs",
      "\nmax: ", amount_of_request_time_in_each_dataset*3, "hrs")

# GitHub only keeps each user event data for 300 events, 
# and with limitation of 100 events per page, request can went up to 3 pages")

min:  3.4785714285714286 hrs 
max:  10.435714285714287 hrs


split all username to 28 dataset, for running script simultaneously:

<ul>
<li>Locally in My Desktop PC,</li>
<li>Google Colab in My Desktop PC,</li>
<li>Locally in My Laptop,</li>
<li>Google Colab in My Laptop</li>
</ul>

Repeating process 7 times in each device, since Google Colab can run for a maximum of 12 hours

GitHub Rest API bottlenecks request limit of user event data with X poll interval for 1 min per requests, meaning I can request for maximum of 60 requests per hr, which is the same amount of request limit for unauthenticated user, so there's no need to use my GitHub token in this process, just have to use different IP address.

In [20]:
split_size = round(amount_of_username_in_each_dataset)
split_list = []
while filtered_username:
    split, filtered_username = filtered_username[:split_size], filtered_username[split_size:]
    split_list.append(split)

In [21]:
len(split_list)

28

In [22]:
file_names = ["username_split"+str(i)+".txt" for i in range(1, 29)]

In [23]:
for i in range(len(file_names)):
    file_name = file_names[i]
    split = split_list[i]
    with open (file_name, "w", newline="", encoding="utf8") as f:
        for username in split:
            f.write(username_dict[username]+"\n")
print("done!")

done!
